# Token Classification. Практическое задание (PJ)

Для закрепления материала модуля вам необходимо решить задачу NER для предоставленного датасета, используя любые доступные вам средства. Модель должна обучаться на файле train.txt, валидироваться на файле dev.txt, а её качество необходимо оценить на файле test.txt.

Для достижения наилучшего результата уделите внимание подбору гиперпарметров как в плане архитектуры, так и в плане обучения модели.

<hr>
**Критерии оценивания проекта:**

- [x] общее качество кода и следование PEP-8;
- [ ] использование рекуррентных сетей;
- [x] использованы варианты архитектур, близкие к state of the art для данной задачи;
- [x] произведен подбор гиперпараметров;
- [x] использованы техники изменения learning rate (lr scheduler);
- [x] использована адекватная задаче функция потерь;
- [x] использованы техники регуляризации;
- [x] корректно проведена валидация модели;
- [ ] использованы техники ensemble;
- [ ] использованы дополнительные данные;
- [x] итоговое значение метрики качества > 0.6 (f1).

<hr>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Прочитаем файлы и посмотрим, сколько какие NER-tags есть в тексте.

In [2]:
with open('train.txt', 'r', encoding="utf-8") as train:
    train_words = train.readlines()
with open('dev.txt', 'r', encoding="utf-8") as dev:
    dev_words = dev.readlines()
with open('test.txt', 'r', encoding="utf-8") as test:
    test_words = test.readlines()

In [3]:
tags = []
for word in train_words:
    if word != '\n':
        token, tag = word.split(' ')
        tags.append(tag.replace('\n', ''))

In [4]:
for word in dev_words:
    if word != '\n':
        token, tag = word.split(' ')
        tags.append(tag.replace('\n', ''))

In [5]:
from collections import Counter
Counter(tags)

Counter({'O': 176027,
         'B-PER': 8469,
         'B-ORG': 6713,
         'I-ORG': 4413,
         'B-LOC': 5729,
         'I-LOC': 1279,
         'I-PER': 5246})

Очевиден дисбаланс классов, но для трансофрмерных моделей не будем здесь ничего делать, они должны справиться без проблем.

В отдельный словарь выведем лейблы ner и присвоенные им цифровые значения. Воспользуемся готовым словарем из интернета, так имеющиеся данные не отличаются от них.

In [6]:
label2id = {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

id2label={0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

label_list = list(label2id.keys())

In [7]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Обработаем загруженные файлы:
- разобъем их на предолжения по знаку \n
- для каждого предложения в отдельный список соберем его токены и ner_tags
- ner_tags заменим на его цифровое значение
- переведем все 3 файла в формате dataset

In [8]:
def split_into_sentences(lst):
    temp_list_tokens = []
    temp_list_ner_tags = []
    res = []
    count = 0
    for word in lst:
        if word == '\n':
            res.append({'id': count,
                        'tokens': temp_list_tokens,
                        'ner_tags': temp_list_ner_tags
                
            })
            temp_list_tokens = []
            temp_list_ner_tags = []
            count += 1
        else:
            token, ner_tag = word.split(' ')
            if token not in label_list:
                temp_list_tokens.append(token)
                temp_list_ner_tags.append(label2id[ner_tag.replace('\n', '')])


    return res

In [9]:
from datasets import Dataset, DatasetDict
import datasets

dataset = DatasetDict({
    "train": Dataset.from_pandas(pd.DataFrame(split_into_sentences(train_words))),
    "valid": Dataset.from_pandas(pd.DataFrame(split_into_sentences(dev_words))),
    'test': Dataset.from_pandas(pd.DataFrame(split_into_sentences(test_words)))
    })
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7746
    })
    valid: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2582
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2582
    })
})

In [10]:
for token, ner_tag in zip(dataset['train'][0]['tokens'], dataset['train'][0]['ner_tags']):
    print(f'{token:_<40}{ner_tag}')

"_______________________________________0
Если____________________________________0
Миронов_________________________________1
занял___________________________________0
столь___________________________________0
оппозиционную___________________________0
позицию_________________________________0
,_______________________________________0
то______________________________________0
мне_____________________________________0
представляется__________________________0
,_______________________________________0
что_____________________________________0
для_____________________________________0
него____________________________________0
было____________________________________0
бы______________________________________0
порядочным______________________________0
и_______________________________________0
правильным______________________________0
уйти____________________________________0
в_______________________________________0
отставку________________________________0
с_________________________________

Все вроде бы хорошо, но есть небольшая техническая трудность, на которую надо обратить внимание при решении подобных задач. При токенизации, например, с помощью алгоритма WordPiece, часть слов может быть разбита на несколько токенов. Количество токенов перестанет совпадать с количеством классов для текста. То же самое касается специальных токенов, таких как [CLS], [SEP] и токенов для паддинга [PAD].

Придется использовать дополнительную функцию для выравнивания длины токенов после токенизатора.

Воспользуюсь русскоязычной моделью https://huggingface.co/DeepPavlov/rubert-base-cased

Посмотрим на пример одного предложения

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [12]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

print(f'Token len from BERT tokenizer__{len(tokens)}')
print(f'Token len initial tokens__{len(example["tokens"])}\n')

print(' '.join(tokens))

Token len from BERT tokenizer__53
Token len initial tokens__47

[CLS] " Если Миронов занял столь оппозиционную позицию , то мне представляется , что для него было бы поряд ##очным и правильным уйти в отставку с заним ##аемого им поста , поста , который предоставлен ему сегодня " Единой Россией ' ' и никем больше ' ' , - заключает Исаев . [SEP]


In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True,
                               remove_columns = ['id', 'tokens', 'ner_tags'])
tokenized_dataset

Map:   0%|          | 0/7746 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2582 [00:00<?, ? examples/s]

Map:   0%|          | 0/2582 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7746
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2582
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2582
    })
})

Посмотрим, как после функции проставились токены:

In [15]:
for token, label in zip(tokenizer.convert_ids_to_tokens(tokenized_dataset['train'][0]['input_ids']), 
                        tokenized_dataset['train'][0]['labels']):
    print(f'{token:_<40}{label}')

[CLS]___________________________________-100
"_______________________________________0
Если____________________________________0
Миронов_________________________________1
занял___________________________________0
столь___________________________________0
оппозиционную___________________________0
позицию_________________________________0
,_______________________________________0
то______________________________________0
мне_____________________________________0
представляется__________________________0
,_______________________________________0
что_____________________________________0
для_____________________________________0
него____________________________________0
было____________________________________0
бы______________________________________0
поряд___________________________________0
##очным_________________________________-100
и_______________________________________0
правильным______________________________0
уйти____________________________________0
в___________________________

Воспользуемся метрикой качества seqeval https://huggingface.co/spaces/evaluate-metric/seqeval

<hr>
**For info:** seqeval is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on.

Overall:
- accuracy: the average accuracy, on a scale between 0.0 and 1.0.
- precision: the average precision, on a scale between 0.0 and 1.0.
- recall: the average recall, on a scale between 0.0 and 1.0.
- f1: the average F1 score, which is the harmonic mean of the precision and recall. It also has a scale of 0.0 to 1.0.
<hr>

Далее 

In [16]:
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [19]:
LR_VALUES = (2e-5, 5e-5)
DECAY_VALUES = (1e-4, 0.1)

In [20]:
from transformers import TrainingArguments, Trainer

for i, learning_rate in enumerate(LR_VALUES):
    for j, weight_decay in enumerate(DECAY_VALUES):
            
            model = AutoModelForTokenClassification.from_pretrained(
                "DeepPavlov/rubert-base-cased", num_labels=9, id2label=id2label, label2id=label2id)
            print(f'Log: training for l_r:{learning_rate}, w_d:{weight_decay}...')
            
            training_args = TrainingArguments(
                output_dir="token_class_model",
                learning_rate=learning_rate,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                num_train_epochs=3,
                weight_decay=weight_decay,
                evaluation_strategy="epoch",
                push_to_hub=False,
                save_strategy="no", 
                group_by_length=True,
                warmup_ratio=0.1,
                optim="adamw_torch",
                lr_scheduler_type="cosine",
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["valid"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )

            trainer.train()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

Log: training for l_r:2e-05, w_d:0.0001...


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.025560,0.952201,0.966999,0.959543,0.992415
2,0.219700,0.027997,0.959257,0.971220,0.965202,0.992886
3,0.015100,0.020558,0.968702,0.973906,0.971297,0.994512


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

...

Log: training for l_r:5e-05, w_d:0.0001...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.030848,0.948133,0.964505,0.956249,0.992082
2,0.148000,0.020419,0.965983,0.975249,0.970594,0.994493
3,0.014700,0.018788,0.971855,0.973906,0.972880,0.995179


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

Log: training for l_r:5e-05, w_d:0.1...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.028757,0.956029,0.971988,0.963943,0.993042
2,0.147900,0.022801,0.966337,0.974866,0.970583,0.994003
3,0.014100,0.018770,0.972828,0.975441,0.974133,0.994983


Итак, в принципе задача NER classification для RuBert на данном датасет оказалась очень легкой. При всех выбранных парах learning_rate + weight decay модель справилась неплохо. Если сравнивать по метрике F1, то немного лучше сработали параметры:

- learning rate = 5e-05
- weight decay = 0.1

Возьмем эти параметры для тестирования на тестовой выборке.

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", num_labels=9, id2label=id2label, label2id=label2id)
print(f'Log: training for l_r:{learning_rate}, w_d:{weight_decay}...')

training_args = TrainingArguments(
    output_dir="token_class_model",
    learning_rate=5e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    push_to_hub=False,
    save_strategy="no", 
    group_by_length=True,
    warmup_ratio=0.1,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

Log: training for l_r:5e-05, w_d:0.1...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.028757,0.956029,0.971988,0.963943,0.993042
2,0.147900,0.022801,0.966337,0.974866,0.970583,0.994003
3,0.014100,0.018770,0.972828,0.975441,0.974133,0.994983


TrainOutput(global_step=1455, training_loss=0.05755561848276669, metrics={'train_runtime': 137.4864, 'train_samples_per_second': 169.02, 'train_steps_per_second': 10.583, 'total_flos': 319955671049400.0, 'train_loss': 0.05755561848276669, 'epoch': 3.0})

In [22]:
predictions = trainer.predict(test_dataset=tokenized_dataset["test"])

Для оценки качества на тестовой выборки посмотрим метрику по итогу работы trainer.predict, а также визуально оценим результаты на нескольких текстах. Для этого воспользуемся pipeline из трансформеров.

In [23]:
predictions.metrics

{'test_loss': 0.017470069229602814,
 'test_precision': 0.9741223307998552,
 'test_recall': 0.9812249362012395,
 'test_f1': 0.9776607337450054,
 'test_accuracy': 0.9954596189863599,
 'test_runtime': 5.2869,
 'test_samples_per_second': 488.373,
 'test_steps_per_second': 30.642}

In [24]:
import torch
device = torch.device ('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
from transformers import pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, device = 0)

In [26]:
ner('Меня зовут Шахова Екатерина и я родом из Москвы, работаю в компании Ромашки')

[{'entity': 'B-PER',
  'score': 0.99680483,
  'index': 3,
  'word': 'Шах',
  'start': 11,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9970294,
  'index': 4,
  'word': '##ова',
  'start': 14,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99362975,
  'index': 5,
  'word': 'Екатерина',
  'start': 18,
  'end': 27},
 {'entity': 'B-LOC',
  'score': 0.9964785,
  'index': 10,
  'word': 'Москвы',
  'start': 41,
  'end': 47},
 {'entity': 'B-PER',
  'score': 0.9969182,
  'index': 16,
  'word': 'Ромаш',
  'start': 68,
  'end': 73},
 {'entity': 'I-PER',
  'score': 0.9354444,
  'index': 17,
  'word': '##ки',
  'start': 73,
  'end': 75}]

In [27]:
ner(dataset['test'][2]['tokens'])

[[],
 [],
 [],
 [{'entity': 'B-ORG',
   'score': 0.58599067,
   'index': 1,
   'word': 'Yahoo',
   'start': 0,
   'end': 5}],
 [],
 [{'entity': 'B-PER',
   'score': 0.60913134,
   'index': 1,
   'word': 'Кэрол',
   'start': 0,
   'end': 5}],
 [{'entity': 'B-PER',
   'score': 0.97244275,
   'index': 1,
   'word': 'Барт',
   'start': 0,
   'end': 4}],
 [],
 [{'entity': 'B-PER',
   'score': 0.96958554,
   'index': 1,
   'word': 'Джерри',
   'start': 0,
   'end': 6}],
 [{'entity': 'B-PER',
   'score': 0.9828484,
   'index': 1,
   'word': 'Янга',
   'start': 0,
   'end': 4}],
 [],
 [],
 [],
 []]

# Финальные выводы:

Задача классификации сущностей оказалась довольно легкой для RuBert, несмотря на серьезный дисбаланс классов. Метрика на тестовом датасете довольно высокая и визуальные примеры показали хороший результаты, хотя компанию Ромашка модель отнесла к персоне, а не к организации.